In [1]:
import numpy as np 
import pandas as pd
from sentence_transformers import SentenceTransformer, util

2023-09-14 16:01:29.370842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 16:01:30.064193: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-14 16:01:34.102957: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64/nvidia:/usr/local/cuda-11.2/lib64:/usr/lib/nvidia:/usr/local/cuda-11.2/lib:/

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [17]:
# can replace with other prompts/occupations
pmpt = 'A photo of the face of a singer'

# example captions, combined with prompts for a few other professions (to use as a comparison)
# can replace with file(s) containing training captions
texts = ['A silhouette of a singer picture image.',
    'Singer with microphone Stock Photo',
    'colour photograph of singer on stage',
    'Close-up picture of singer sitting toward a microphone',
    'Portrait of a singer recording a track — Stock Photo',
    'Carrie Underwood, Country Singer',
    'Rock Singer Patti Smith In Concert Acrylic Print by George Rose',
    'bollywood singer wearing sunglasses',
    'artist male guitarist guitar acoustic playing sitting singing song singer performance performer musician concert',
    'The Wonder Singer',
    'Image of female singer',
    'Singer recording a song in studio',
    'Singer Sewing Machine Head From 1879',
    'Wedding Singer and Musician - Guitar and/or Vocals',
    'Singer and actor Frank Sinatra poses for a portrait to promote his NBC Radio show circa 1946 in New York City /Getty Images',
    'Full length portrait of a male singer performing a song',
    'A photo of the face of a doctor',
    'A photo of the face of a musician',
    'A photo of the face of an assistant',
    'A photo of the face of an engineer',
    'A photo of the face of a politician']

In [12]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [18]:
embeddings1 = sbert_model.encode(texts, convert_to_tensor=True) # encode captions
embeddings2 = sbert_model.encode([pmpt], convert_to_tensor=True) # encode prompts

# compute cosine similarity between prompts and captions
cosine_scores = util.cos_sim(embeddings1, embeddings2).detach().cpu().numpy()

# sort according to cosine sim. values (descending) and get corresponding indices
indices = cosine_scores[:,0].argsort()[::-1]

ordered_texts = [texts[i] for i in indices] 

for i in indices:
    print(texts[i])
    print('score:', cosine_scores[i,0])
    print('**************')

A silhouette of a singer picture image.
score: 0.850815
**************
A photo of the face of a musician
score: 0.8463416
**************
Singer with microphone Stock Photo
score: 0.7935266
**************
Portrait of a singer recording a track — Stock Photo
score: 0.7829431
**************
Image of female singer
score: 0.7622549
**************
Close-up picture of singer sitting toward a microphone
score: 0.7597242
**************
Full length portrait of a male singer performing a song
score: 0.72513187
**************
colour photograph of singer on stage
score: 0.7205446
**************
A photo of the face of a politician
score: 0.6968361
**************
A photo of the face of a doctor
score: 0.61280626
**************
A photo of the face of an assistant
score: 0.5931772
**************
A photo of the face of an engineer
score: 0.5772126
**************
Singer recording a song in studio
score: 0.574843
**************
Rock Singer Patti Smith In Concert Acrylic Print by George Rose
score: 0.49014

## Observations
+ Structure/phrasing seems to be very important for similarity. For example, texts that follow the structure "A photo of the face of a/an [OCCUPATION]" (where the occupation is not singer) have higher similarity scores than several texts that pertain to the occupation singer (e.g. "Singer recording a song in studio").